In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(r'../data')
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
def function_that_warns():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    function_that_warns()  # this will not show a warning

C:\Users\Rushi.Soni\AppData\Local\Continuum\anaconda3\envs\ocr_mds_expense\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from flair.data import Sentence
from flair.models import MultiTagger
from flair.tokenization import SciSpacyTokenizer
# load biomedical tagger
tagger = MultiTagger.load("hunflair")

2022-01-20 13:06:28,069 loading file C:\Users\Rushi.Soni\.flair\models\hunflair-celline-v1.0.pt
2022-01-20 13:06:37,870 loading file C:\Users\Rushi.Soni\.flair\models\hunflair-chemical-full-v1.0.pt
2022-01-20 13:06:48,392 loading file C:\Users\Rushi.Soni\.flair\models\hunflair-disease-full-v1.0.pt
2022-01-20 13:06:59,355 loading file C:\Users\Rushi.Soni\.flair\models\hunflair-gene-full-v1.0.pt
2022-01-20 13:07:11,576 loading file C:\Users\Rushi.Soni\.flair\models\hunflair-species-full-v1.1.pt


In [3]:
ClinNotes_df = pd.read_csv('ClinNotes.csv')
print(ClinNotes_df.columns, ClinNotes_df.shape)

Index(['category', 'notes'], dtype='object') (818, 2)


In [4]:
def get_disease_chemical(clinical_note_text):
    sentence = Sentence(clinical_note_text, use_tokenizer=SciSpacyTokenizer())
    tagger.predict(sentence)
    disease = [disease.text for disease in sentence.get_spans("hunflair-disease")]
    chemical = [chemical.text for chemical in sentence.get_spans("hunflair-chemical")]
    return disease, chemical

In [5]:
ClinNotes_df= ClinNotes_df.sample(100).reset_index(drop=True)
ClinNotes_df['entity_tagging'] = ClinNotes_df['notes'].progress_apply(get_disease_chemical)
ClinNotes_df.head()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [1:06:51<00:00, 40.12s/it]


,category,notes,entity_tagging
0,Cardiovascular / Pulmonary,"REASON FOR CONSULTATION: , Pulmonary embolism....","([syncopal episode, shortness of breath, chest..."
1,Cardiovascular / Pulmonary,"PREOPERATIVE DIAGNOSIS:, Airway stenosis with...","([Airway stenosis, Airway stenosis, bleeding, ..."
2,Cardiovascular / Pulmonary,"PROCEDURE DONE: ,Resting Myoview and adenosin...","([dizziness, shortness of breath, arrhythmias]..."
3,Gastroenterology,"PREPROCEDURE DIAGNOSIS:, Change in bowel func...","([hematochezia, ulcerations, edema, mucosal ab..."
4,Neurology,"HISTORY OF PRESENT ILLNESS: ,The patient is a...","([right arm pain, pain, dull pain, achy sensat..."


In [6]:
ClinNotes_df['disease'] = ClinNotes_df['entity_tagging'].apply(lambda x: x[0])
ClinNotes_df['chemical'] = ClinNotes_df['entity_tagging'].apply(lambda x: x[1])
del ClinNotes_df['entity_tagging']

In [7]:
print(ClinNotes_df.shape, ClinNotes_df.columns)

(100, 4) Index(['category', 'notes', 'disease', 'chemical'], dtype='object')


In [8]:
ClinNotes_df.head()

,category,notes,disease,chemical
0,Cardiovascular / Pulmonary,"REASON FOR CONSULTATION: , Pulmonary embolism....","[syncopal episode, shortness of breath, chest ...","[Aspirin.,2 . Potassium.,3 . Lasix.,4 . ..."
1,Cardiovascular / Pulmonary,"PREOPERATIVE DIAGNOSIS:, Airway stenosis with...","[Airway stenosis, Airway stenosis, bleeding, t...",[silicon]
2,Cardiovascular / Pulmonary,"PROCEDURE DONE: ,Resting Myoview and adenosin...","[dizziness, shortness of breath, arrhythmias]","[adenosine, Tc-99 m tetrofosmin, adenosine, ad..."
3,Gastroenterology,"PREPROCEDURE DIAGNOSIS:, Change in bowel func...","[hematochezia, ulcerations, edema, mucosal abn...",[]
4,Neurology,"HISTORY OF PRESENT ILLNESS: ,The patient is a...","[right arm pain, pain, dull pain, achy sensati...","[Medrol, ACTOplus, albuterol, AndroGel, Asteli..."


In [9]:
ClinNotes_df.to_pickle('ClinNotes_entity_df')